In [19]:
import langdetect
import matplotlib.pyplot
import nltk
import numpy
import pandas
import pyLDAvis
import pyLDAvis.sklearn
import regex
import sklearn

In [8]:
path = './Activity15-Activity17/latimeshealth.txt'
df = pandas.read_csv(path, sep="|", header=None)
df.columns = ["id", "datetime", "tweettext"]

In [3]:
%matplotlib inline

In [9]:
def dataframe_quick_look(df, nrows):
    print("SHAPE:\n{shape}\n".format(shape=df.shape))
    print("COLUMN NAMES:\n{names}\n".format(names=df.columns))
    print("HEAD:\n{head}\n".format(head=df.head(nrows)))
dataframe_quick_look(df, nrows=2)

SHAPE:
(4171, 3)

COLUMN NAMES:
Index(['id', 'datetime', 'tweettext'], dtype='object')

HEAD:
                   id                        datetime  \
0  576760256031682561  Sat Mar 14 15:02:15 +0000 2015   
1  576715414811471872  Sat Mar 14 12:04:04 +0000 2015   

                                           tweettext  
0  Five new running shoes that aim to go the extr...  
1  Gym Rat: Disq class at Crunch is intense worko...  



In [10]:
df.head()

,id,datetime,tweettext
0,576760256031682561,Sat Mar 14 15:02:15 +0000 2015,Five new running shoes that aim to go the extr...
1,576715414811471872,Sat Mar 14 12:04:04 +0000 2015,Gym Rat: Disq class at Crunch is intense worko...
2,576438353555365888,Fri Mar 13 17:43:07 +0000 2015,Noshing through thousands of ideas at Natural ...
3,576438347003908096,Fri Mar 13 17:43:06 +0000 2015,"Natural Products Expo also explores beauty, su..."
4,576413058177712128,Fri Mar 13 16:02:36 +0000 2015,Free Fitness Weekends in South Bay beach citie...


In [11]:
raw = df['tweettext'].tolist()
print("HEADLINES:\n{lines}\n".format(lines=raw[:5]))
print("LENGTH:\n{length}\n".format(length=len(raw)))

HEADLINES:
['Five new running shoes that aim to go the extra mile http://lat.ms/1ELp3wU', 'Gym Rat: Disq class at Crunch is intense workout on pulley system http://lat.ms/1EKOFdr', 'Noshing through thousands of ideas at Natural Products Expo West http://lat.ms/1EHqywg', 'Natural Products Expo also explores beauty, supplements and more http://lat.ms/1EHqyfE', 'Free Fitness Weekends in South Bay beach cities aim to spark activity http://lat.ms/1EH3SMC']

LENGTH:
4171



In [12]:
def do_language_identifying(txt):
    try:
        the_language = langdetect.detect(txt)
    except:
        the_language = 'none'
    return the_language


def do_lemmatizing(wrd):
    out = nltk.corpus.wordnet.morphy(wrd)
    return (wrd if out is None else out)


def do_tweet_cleaning(txt):
# identify language of tweet
# return null if language not english
    lg = do_language_identifying(txt)
    if lg != 'en':
        return None

# split the string on whitespace
    out = txt.split(' ')

# identify screen names
# replace with SCREENNAME
    out = ['SCREENNAME' if i.startswith('@') else i for i in out]

# identify urls
# replace with URL
    out = ['URL' if bool(regex.search('http[s]?://', i)) else i for i in out]

# remove all punctuation
    out = [regex.sub('[^\\w\\s]|\n', '', i) for i in out]

# make all non-keywords lowercase
    keys = ['SCREENNAME', 'URL']
    out = [i.lower() if i not in keys else i for i in out]

# remove keywords
    out = [i for i in out if i not in keys]

# remove stopwords
    list_stop_words = nltk.corpus.stopwords.words('english')
    list_stop_words = [    regex.sub('[^\\w\\s]', '', i) for i in list_stop_words]

    out = [i for i in out if i not in list_stop_words]

# lemmatizing
    out = [do_lemmatizing(i) for i in out]

# keep words 4 or more characters long
    out = [i for i in out if len(i) >= 5]

    return out

In [13]:
clean = list(map(do_tweet_cleaning, raw))

In [15]:
clean = list(filter(None.__ne__, clean))
print("HEADLINES:\n{lines}\n".format(lines=clean[:5]))
print("LENGTH:\n{length}\n".format(length=len(clean)))

HEADLINES:
[['running', 'shoes', 'extra'], ['class', 'crunch', 'intense', 'workout', 'pulley', 'system'], ['thousand', 'natural', 'product'], ['natural', 'product', 'explore', 'beauty', 'supplement'], ['fitness', 'weekend', 'south', 'beach', 'spark', 'activity']]

LENGTH:
4100



In [16]:
clean_sentences = [" ".join(i) for i in clean]
print(clean_sentences[0:10])

['running shoes extra', 'class crunch intense workout pulley system', 'thousand natural product', 'natural product explore beauty supplement', 'fitness weekend south beach spark activity', 'kayla harrison sacrifice', 'sonic treatment alzheimers disease', 'ultrasound brain restore memory alzheimers needle onlyso farin mouse', 'apple researchkit really medical research', 'warning chantix drink taking might remember']


In [18]:
number_words = 10
number_docs = 10
number_features = 1000

In [20]:
vectorizer1 = sklearn.feature_extraction.text.CountVectorizer(
    analyzer="word",
    max_df=0.95, 
    min_df=10, 
    max_features=number_features
)
clean_vec1 = vectorizer1.fit_transform(clean_sentences)
print(clean_vec1[0])

feature_names_vec1 = vectorizer1.get_feature_names()

  (0, 321)	1


In [21]:
def perplexity_by_ntopic(data, ntopics):
    output_dict = {
        "Number Of Topics": [], 
        "Perplexity Score": []
    }
    for t in ntopics:
        lda = sklearn.decomposition.LatentDirichletAllocation(
            n_components=t,
            learning_method="online",
            random_state=0
        )
        lda.fit(data)
        output_dict["Number Of Topics"].append(t)
        output_dict["Perplexity Score"].append(lda.perplexity(data))
    output_df = pandas.DataFrame(output_dict)
    index_min_perplexity = output_df["Perplexity Score"].idxmin()
    output_num_topics = output_df.loc[
        index_min_perplexity,  # index
        "Number Of Topics"  # column
    ]
    return (output_df, output_num_topics)
df_perplexity, optimal_num_topics = perplexity_by_ntopic(
    clean_vec1, 
    ntopics=[i for i in range(1, 21) if i % 2 == 0]
)
print(df_perplexity)


   Number Of Topics  Perplexity Score
0                 2        356.367714
1                 4        412.411161
2                 6        437.894952
3                 8        463.372176
4                10        487.684619
5                12        493.664355
6                14        517.631987
7                16        516.598648
8                18        541.126746
9                20        543.837987


In [22]:
lda = sklearn.decomposition.LatentDirichletAllocation(
    n_components=optimal_num_topics,
    learning_method="online",
    random_state=0
)
lda.fit(clean_vec1)

LatentDirichletAllocation(learning_method='online', n_components=2,
                          random_state=0)

In [23]:
def get_topics(mod, vec, names, docs, ndocs, nwords):
    # word to topic matrix
    W = mod.components_
    W_norm = W / W.sum(axis=1)[:, numpy.newaxis]
    # topic to document matrix
    H = mod.transform(vec)
    W_dict = {}
    H_dict = {}
    for tpc_idx, tpc_val in enumerate(W_norm):
        topic = "Topic{}".format(tpc_idx)
        # formatting w
        W_indices = tpc_val.argsort()[::-1][:nwords]
        W_names_values = [
            (round(tpc_val[j], 4), names[j]) 
            for j in W_indices
        ]
        W_dict[topic] = W_names_values
        # formatting h
        H_indices = H[:, tpc_idx].argsort()[::-1][:ndocs]
        H_names_values = [
        (round(H[:, tpc_idx][j], 4), docs[j]) 
            for j in H_indices
        ]
        H_dict[topic] = H_names_values
    W_df = pandas.DataFrame(
        W_dict, 
        index=["Word" + str(i) for i in range(nwords)]
    )
    H_df = pandas.DataFrame(
        H_dict,
        index=["Doc" + str(i) for i in range(ndocs)]
    )
    return (W_df, H_df)

W_df, H_df = get_topics(
    mod=lda,
    vec=clean_vec1,
    names=feature_names_vec1,
    docs=raw,
    ndocs=number_docs, 
    nwords=number_words
)

print(W_df)

                     Topic0                Topic1
Word0        (0.062, study)      (0.0382, latfit)
Word1      (0.0213, weight)      (0.0347, cancer)
Word2       (0.0205, could)      (0.0309, health)
Word3       (0.0194, brain)       (0.0265, study)
Word4       (0.016, report)      (0.0223, people)
Word5       (0.0153, child)     (0.0176, patient)
Word6    (0.015, scientist)  (0.0164, researcher)
Word7  (0.0144, california)       (0.0163, woman)
Word8      (0.0133, common)       (0.0159, death)
Word9     (0.0126, medical)      (0.0156, doctor)


In [24]:
print(H_df)

                                                 Topic0  \
Doc0  (0.9373, Many animals that suffer from contagi...   
Doc1  (0.936, You can't trust your drug dealer, rese...   
Doc2  (0.9359, Sun protection is a fashion statement...   
Doc3  (0.9356, Some parts of your body "age" faster ...   
Doc4  (0.9348, Flu shot benefit for adults isn't big...   
Doc5  (0.9336, Rotator cuff exercise uses dumbbell h...   
Doc6                        (0.9283, @ainavar1 Agreed!)   
Doc7  (0.9282, Sentinel chickens form a simple and e...   
Doc8  (0.9269, New documentary "All of Me" explores ...   
Doc9  (0.9268, “No one should assume e-cigarettes ar...   

                                                 Topic1  
Doc0  (0.9412, Researchers find 2 types of brain atr...  
Doc1                (0.938, @mypaleoplan Looks delish!)  
Doc2  (0.9372, Is your husband faithful? If so, than...  
Doc3  (0.937, Starbucks says it will post calorie co...  
Doc4  (0.9356, Food as medicine? What to make of the...  
Do

In [25]:
lda_plot = pyLDAvis.sklearn.prepare(lda, clean_vec1, vectorizer1, R=10)
pyLDAvis.display(lda_plot)